In [1]:
%pip install arxiv

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6105 sha256=a3e269430fa8d25a8f20ec473f300d99ac947515ef258f56581affb9e1767ed0
  Stored in directory: c:\users\samme\appdata\local\pip\cache\wheels\3b\25\2a\105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k

   ------------- -------------------------- 1/3 [feedparser]
   ------------- -------------------------- 1/3 [feedparser]
   ------------- -------------------------- 1/3 [feedparser]
   ------------- -------------------------- 1/3 [feedparser]
   ------------- -------------------------- 1/3 [feedparser]
   ------------- -------------------------- 1/3 [feedparser]
   ---------------------------------------- 3/3 [arxiv]

Note: you may need to restart the kernel to use updated packages.


  DEPRECATION: Building 'sgmllib3k' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'sgmllib3k'. Discussion can be found at https://github.com/pypa/pip/issues/6334


1st Retriver tool

In [4]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


In [16]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki_tool=WikipediaQueryRun(api_wrapper=api_wrapper)

In [17]:
wiki_tool.name

'wikipedia'

2nd Reteriver tool

In [9]:
from langchain.document_loaders import WebBaseLoader
from langchain.vectorstores import FAISS
from langchain_openai import OpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings


loader = WebBaseLoader("https://docs.langchain.com/langsmith/home")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
embeddings=SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')

vectordb=FAISS.from_documents(documents,embedding=embeddings)

retriever=vectordb.as_retriever()
retriever

C:\Users\Samme\AppData\Local\Temp\ipykernel_21244\1049955194.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')


VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002E78CC2B590>, search_kwargs={})

To make retriver as CreateRetriverTool

In [12]:
from langchain.tools.retriever import create_retriever_tool

retriver_tool=create_retriever_tool(retriever,"name=langsmith_search","description=Search for the information about Langsmith. For any question about langsmith ask this tool")

In [13]:
retriver_tool

Tool(name='name=langsmith_search', description='description=Search for the information about Langsmith. For any question about langsmith ask this tool', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000002E78A175800>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002E78CC2B590>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x000002E78A175A80>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002E78CC2B590>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'],

3rd Reteriver Tool

In [15]:
# Arxiv Tool

from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

Let's combine this 3 tools

In [18]:
tools=[wiki_tool,retriver_tool,arxiv]

In [19]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\Samme\\Git\\Langchain_bot\\env311\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='name=langsmith_search', description='description=Search for the information about Langsmith. For any question about langsmith ask this tool', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000002E78A175800>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002E78CC2B590>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_document

In [20]:
from dotenv import load_dotenv
load_dotenv()
import os
os.environ['OPENAI_API_KEY']=os.getenv('OPENAI_API_KEY')
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="mistralai/mistral-small-3.1-24b-instruct:free",
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    temperature=0.7
)

In [24]:
from langchain import hub
# Get the prompt to use - you can modify this!
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

Query from the specific tool and that is where we are using agents
Agent will search the given query in all the tools and give the output whereever he will get the result 

In [25]:
#Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

In [27]:
# Agent Executer
from langchain.agents import AgentExecutor
agent_executer=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executer

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [30]:
agent_executer.invoke({"input":"tell me about Langsmith"})



> Entering new AgentExecutor chain...
Langsmith is a platform designed to help developers and organizations manage, evaluate, and iterate on large language models (LLMs) and other AI models. It provides a suite of tools to streamline the process of working with AI, from training and fine-tuning models to deploying and monitoring them in production. Here are some key features and aspects of Langsmith:

1. **Model Management**: Langsmith allows users to manage multiple models, track their versions, and maintain a centralized repository of AI assets. This is crucial for organizations that work with numerous models and need to keep track of their performance and updates.

2. **Evaluation and Testing**: The platform offers tools for evaluating models on various metrics and datasets. This helps in understanding the strengths and weaknesses of different models and in making data-driven decisions for improvement.

3. **Experiment Tracking**: Langsmith provides features to track experiments, 

{'input': 'tell me about Langsmith',
 'output': 'Langsmith is a platform designed to help developers and organizations manage, evaluate, and iterate on large language models (LLMs) and other AI models. It provides a suite of tools to streamline the process of working with AI, from training and fine-tuning models to deploying and monitoring them in production. Here are some key features and aspects of Langsmith:\n\n1. **Model Management**: Langsmith allows users to manage multiple models, track their versions, and maintain a centralized repository of AI assets. This is crucial for organizations that work with numerous models and need to keep track of their performance and updates.\n\n2. **Evaluation and Testing**: The platform offers tools for evaluating models on various metrics and datasets. This helps in understanding the strengths and weaknesses of different models and in making data-driven decisions for improvement.\n\n3. **Experiment Tracking**: Langsmith provides features to trac

In [31]:
agent_executer.invoke({"input":"tell me about Machine Learning"})



> Entering new AgentExecutor chain...
Machine Learning (ML) is a subset of artificial intelligence (AI) that focuses on the development of algorithms and statistical models that enable computers to perform specific tasks without using explicit instructions. Instead of being programmed with exact rules, machine learning models learn patterns from data and make decisions or predictions based on that learning. Here are some key aspects of machine learning:

### Types of Machine Learning

1. **Supervised Learning**:
   - **Definition**: The model is trained on a labeled dataset, meaning that each training example is paired with an output label.
   - **Examples**: Classification (e.g., spam detection, image classification) and regression (e.g., predicting house prices, stock prices).
   - **Common Algorithms**: Linear regression, logistic regression, decision trees, support vector machines (SVM), and neural networks.

2. **Unsupervised Learning**:
   - **Definition**: The model is given d

{'input': 'tell me about Machine Learning',
 'output': "Machine Learning (ML) is a subset of artificial intelligence (AI) that focuses on the development of algorithms and statistical models that enable computers to perform specific tasks without using explicit instructions. Instead of being programmed with exact rules, machine learning models learn patterns from data and make decisions or predictions based on that learning. Here are some key aspects of machine learning:\n\n### Types of Machine Learning\n\n1. **Supervised Learning**:\n   - **Definition**: The model is trained on a labeled dataset, meaning that each training example is paired with an output label.\n   - **Examples**: Classification (e.g., spam detection, image classification) and regression (e.g., predicting house prices, stock prices).\n   - **Common Algorithms**: Linear regression, logistic regression, decision trees, support vector machines (SVM), and neural networks.\n\n2. **Unsupervised Learning**:\n   - **Definiti

In [32]:
agent_executer.invoke({"input":"What's the paper 1605.08386 about?"})



> Entering new AgentExecutor chain...
The paper titled "1605.08386" is a preprint that was uploaded to the arXiv repository. The specific details of the paper can vary, but as of my knowledge cutoff in October 2023, I can provide a general approach to finding out about it.

To determine the content of arXiv:1605.08386, you can follow these steps:

1. **Visit arXiv.org**: Go to the arXiv website, which hosts a large collection of preprints in various fields, particularly in physics, mathematics, computer science, and other scientific disciplines.

2. **Search for the Paper**: Use the search function on arXiv.org and enter the identifier "1605.08386". This should bring up the specific paper you are interested in.

3. **Read the Abstract and Introduction**: The abstract and introduction sections of the paper will provide a summary of the research, its objectives, methods, and key findings.

4. **Check the Authors and Affiliations**: This can give you context about the background and exp

{'input': "What's the paper 1605.08386 about?",
 'output': 'The paper titled "1605.08386" is a preprint that was uploaded to the arXiv repository. The specific details of the paper can vary, but as of my knowledge cutoff in October 2023, I can provide a general approach to finding out about it.\n\nTo determine the content of arXiv:1605.08386, you can follow these steps:\n\n1. **Visit arXiv.org**: Go to the arXiv website, which hosts a large collection of preprints in various fields, particularly in physics, mathematics, computer science, and other scientific disciplines.\n\n2. **Search for the Paper**: Use the search function on arXiv.org and enter the identifier "1605.08386". This should bring up the specific paper you are interested in.\n\n3. **Read the Abstract and Introduction**: The abstract and introduction sections of the paper will provide a summary of the research, its objectives, methods, and key findings.\n\n4. **Check the Authors and Affiliations**: This can give you contex